<h1><font color="#f37626">[Lifecycle]</font> add_expriment 예제 코드</h1>

-----

`add_experiment`는 모델링 과정 중 실험 이력으로 저장하고자 하는 모델을 자동으로 accuinsight+ modeler 콘솔 화면에 기록해주는 메소드 입니다.

- data: breast cancer data 
    - label: (1:악성) / (0: 양성)
- sklearn 사용
- logistic regression
---

### 1. accuinsight() 객체 생성
- sklearn을 사용하므로, ML 모듈의 accuinsight 메소드를 불러와야 함

In [1]:
from Accuinsight.Lifecycle.ML import accuinsight

accu = accuinsight()

#### [optional] 1-(1) alarm 설정 

- `send_message()`로 alarm message를 push할 수 있습니다.
- `send_message(message = 'your_message')`  

- ML Lifecycle의 경우 threshold를 활용한 message push 서비스가 제공되지 않습니다.

- `send_message()`의 message 인수는 list 또는 str 형태여야 합니다. 여러 message를 동시에 발송할 땐 list, 단일 message는 str 형태로 입력합니다.
- `unset_message()`를 사용하면 등록된 message가 삭제됩니다. 

In [2]:
accu.send_message(message='[ML-binary-classification-xgboost] 모델 학습 완료')

#### [optional] 1-(2) slack 설정

- `set_slack()`을 사용하여 특정 slack 채널에도 message를 전달할 수 있습니다. 여러 채널에 발송하고 싶다면, url을 콤마(,)로 구분하여 입력하면 됩니다. message와 유사하게, `unset_slack()`을 사용하면 slack alarm이 해제됩니다.
- slack에 message를 전달하려면, 해당 채널 또는 DM의 webhook url이 필요합니다.

In [3]:
accu.set_slack(hook_url='webhook url')

### 2. 모델 구성

In [3]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression

# Import dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=42)

logistic = LogisticRegression(random_state = 0)
logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [4]:
logistic.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

### 3. 평가지표 및 예측

In [5]:
from sklearn.metrics import accuracy_score, f1_score

pred_test = logistic.predict(X_test)

In [6]:
accuracy = accuracy_score(y_test, pred_test)
f1score = f1_score(y_test, pred_test)

### 4. add experiment

__Data drift 기능을 사용할 경우, `model_monitor=True` 옵션을 사용하여 피처 중요도를 저장합니다.__  
비정형데이터의 경우 사용할 수 없습니다.

In [8]:
with accu.add_experiment(logistic, X_train, y_train, X_test, y_test, model_monitor=True) as exp:
    exp.log_params('max_iter')
    exp.log_metrics('Accuracy', accuracy)
    exp.log_metrics('F1-score', f1score)
    exp.log_tag('binary-breast-cancer-logistic')

### 5. 저장된 모델 불러오기
- `add_experiment()`를 사용하여 모델의 학습 이력을 Lifecycle에 기록할 경우, 자동으로 모델 파일이 저장됩니다.
- 저장된 모델을 불러와 공동 작업자들과 모델을 공유하거나, 모델 재학습을 수행할 수 있습니다.

    1. Accuinsight+ workspace list 혹은 해당 모델의 상세화면으로 접속하여 _Experiment_ 중 불러오고자 하는 모델의 __Run name__을 복사합니다.
    2. ___utils___에서 `load_model()` 함수를 호출하여 모델을 불러올 수 있습니다.

In [9]:
from Accuinsight.Lifecycle.utils import load_model

loaded = load_model('LogisticRegression-98FAE7BEE24D412DAF2CB2E6E00D0BCC_44')

In [10]:
loaded.coef_

array([[ 0.60651942,  0.55577977,  0.32524395, -0.02103228, -0.01494832,
        -0.0944371 , -0.14419526, -0.05941119, -0.03624748, -0.00439298,
         0.03403868,  0.26351517,  0.10136707, -0.12417901, -0.00175519,
        -0.02198481, -0.03251225, -0.00826842, -0.00685513, -0.00163949,
         0.68678762, -0.55335594, -0.20996113, -0.0083787 , -0.03342834,
        -0.30755839, -0.40193348, -0.11774485, -0.10446085, -0.02446907]])